In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings("ignore")

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, accuracy, report, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def svm_nl(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='rbf', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def naive_bayes(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def decision_tree(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random_forest(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def select_k_best(indep_X, dep_Y, k):
    selector = SelectKBest(score_func=chi2, k=k)
    X_new = selector.fit_transform(indep_X, dep_Y)
    selected_columns = indep_X.columns[selector.get_support()].tolist()
    return X_new, selected_columns

def create_results_dataframe(*accuracies):
    index = ['ChiSquare']
    columns = ['Logistic', 'SVM Linear', 'SVM RBF', 'KNN', 'Naive Bayes', 'Decision Tree', 'Random Forest']
    df = pd.DataFrame([accuracies], index=index, columns=columns)
    return df

# Load the dataset
dataset = pd.read_csv("prep.csv")

# Convert categorical data into numerical data
dataset = pd.get_dummies(dataset, drop_first=True)

# Ensure 'classification_yes' is a target column
if 'classification_yes' not in dataset.columns:
    raise ValueError("Target column 'classification_yes' not found in dataset")

indep_X = dataset.drop('classification_yes', axis=1)
dep_Y = dataset['classification_yes']

# Feature selection
kbest_X, selected_columns = select_k_best(indep_X, dep_Y, k=4)

# Split and scale data
X_train, X_test, y_train, y_test = split_scalar(kbest_X, dep_Y)

# Train models and store accuracies
accuracies = [
    logistic(X_train, y_train, X_test, y_test)[1],
    svm_linear(X_train, y_train, X_test, y_test)[1],
    svm_nl(X_train, y_train, X_test, y_test)[1],
    knn(X_train, y_train, X_test, y_test)[1],
    naive_bayes(X_train, y_train, X_test, y_test)[1],
    decision_tree(X_train, y_train, X_test, y_test)[1],
    random_forest(X_train, y_train, X_test, y_test)[1]
]

# Create results dataframe
result_df = create_results_dataframe(*accuracies)
print(result_df)


           Logistic  SVM Linear  SVM RBF   KNN  Naive Bayes  Decision Tree  \
ChiSquare      0.85        0.82     0.83  0.86         0.79           0.89   

           Random Forest  
ChiSquare           0.89  


In [2]:
selected_columns

['bgr', 'bu', 'sc', 'wc']